In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv("7282_1.csv", header=0, encoding='utf8')
train = train[pd.notnull(train["reviews.text"])]
train=train.reset_index(drop=True)
# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, \n" % (train["reviews.text"].size))

Read 35890 labeled train reviews, 



/home/rl/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):

    raw_sentences = tokenizer.tokenize(review.strip())

    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))

    return sentences

In [5]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["reviews.text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/rl/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/rl/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/home/rl/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/rl/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://ihg.co/V4S29" looks like a URL. Beautiful Soup is not

In [6]:
 # Check how many sentences we have in total - should be around 850,000+
print len(sentences)
print sentences[0]

138241
[u'pleasant', u'min', u'walk', u'along', u'the', u'sea', u'front', u'to', u'the', u'water', u'bus']


In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 120  # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


model.init_sims(replace=True)


model_name = "300features_40minwords_10context"
model.save(model_name)

2017-11-09 14:53:39,604 : INFO : collecting all words and their counts
2017-11-09 14:53:39,607 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-09 14:53:39,681 : INFO : PROGRESS: at sentence #10000, processed 120170 words, keeping 7986 word types


Training model...


2017-11-09 14:53:39,821 : INFO : PROGRESS: at sentence #20000, processed 241786 words, keeping 10826 word types
2017-11-09 14:53:39,997 : INFO : PROGRESS: at sentence #30000, processed 364821 words, keeping 13374 word types
2017-11-09 14:53:40,120 : INFO : PROGRESS: at sentence #40000, processed 488391 words, keeping 16738 word types
2017-11-09 14:53:40,234 : INFO : PROGRESS: at sentence #50000, processed 609847 words, keeping 18324 word types
2017-11-09 14:53:40,410 : INFO : PROGRESS: at sentence #60000, processed 733794 words, keeping 20429 word types
2017-11-09 14:53:40,689 : INFO : PROGRESS: at sentence #70000, processed 854583 words, keeping 21889 word types
2017-11-09 14:53:40,815 : INFO : PROGRESS: at sentence #80000, processed 974476 words, keeping 23298 word types
2017-11-09 14:53:40,935 : INFO : PROGRESS: at sentence #90000, processed 1092967 words, keeping 24653 word types
2017-11-09 14:53:41,042 : INFO : PROGRESS: at sentence #100000, processed 1210201 words, keeping 25804 

In [11]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [13]:
model.most_similar("man")

[(u'gentleman', 0.8386572003364563),
 (u'lady', 0.8232420682907104),
 (u'woman', 0.8022923469543457),
 (u'girl', 0.7899739742279053),
 (u'guy', 0.764091968536377),
 (u'receptionist', 0.7487019300460815),
 (u'attendant', 0.7397534251213074),
 (u'owner', 0.7303192615509033),
 (u'clerk', 0.7267169952392578),
 (u'employee', 0.7065222859382629)]

In [15]:
model.most_similar("location")

[(u'proximity', 0.675235390663147),
 (u'neighborhood', 0.6327090263366699),
 (u'downtown', 0.6237961053848267),
 (u'located', 0.617552638053894),
 (u'value', 0.607701301574707),
 (u'venue', 0.5929969549179077),
 (u'town', 0.5890241861343384),
 (u'interstate', 0.5826400518417358),
 (u'spot', 0.5715011954307556),
 (u'convenience', 0.5645418763160706)]

In [16]:
model.most_similar("price")

[(u'rate', 0.7875528335571289),
 (u'cost', 0.6930409669876099),
 (u'value', 0.6706050038337708),
 (u'pricing', 0.6561794877052307),
 (u'prices', 0.5955599546432495),
 (u'rates', 0.5929273366928101),
 (u'money', 0.5635597705841064),
 (u'priced', 0.5359944105148315),
 (u'bargain', 0.49203598499298096),
 (u'overall', 0.48928511142730713)]

In [17]:
model.most_similar("service")

[(u'hospitality', 0.4894063472747803),
 (u'driver', 0.4685381054878235),
 (u'concierge', 0.4596795439720154),
 (u'staff', 0.45761263370513916),
 (u'pricing', 0.43609046936035156),
 (u'food', 0.43171125650405884),
 (u'services', 0.4260099530220032),
 (u'crew', 0.41115158796310425),
 (u'chef', 0.40882542729377747),
 (u'value', 0.39425352215766907)]

In [18]:
model.most_similar("awful")

[(u'terrible', 0.7821382284164429),
 (u'horrible', 0.7735625505447388),
 (u'okay', 0.7328379154205322),
 (u'funny', 0.6945334672927856),
 (u'disgusting', 0.6840559840202332),
 (u'unpleasant', 0.6731683015823364),
 (u'acceptable', 0.6700491905212402),
 (u'odd', 0.6647114753723145),
 (u'gross', 0.6554749011993408),
 (u'poor', 0.6524474024772644)]